In [32]:
import pandas as pd
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import os

In [3]:
directory = "data/SMNI_CMI_TRAIN/Train/"
filenames = os.listdir(directory)
all_frames = []
for name in filenames:
    temp = pd.read_csv(directory+name, header=0)
    all_frames.append(temp)

# all_data = pd.concat(all_frames)
# all_data.head(5)
# all_frames.head()

In [7]:
# S1 obj: one picture
# S2 match: two identical pictures
# S2 nomatch: two different pictures
# all_data.nunique()
# all_frames.nunique()
all_frames[0].columns

Index(['Unnamed: 0', 'trial number', 'sensor position', 'sample num',
       'sensor value', 'subject identifier', 'matching condition', 'channel',
       'name', 'time'],
      dtype='object')

In [10]:
def makeOneRow(df):
    columns = ["trial number", "subject identifier", "name", "matching condition"]
    values = {}
    for sensorPos, sampleNum, sensorVal in zip(df["sensor position"], df["sample num"], df["sensor value"]):
        values[sensorPos+str(sampleNum)] = [sensorVal]
    for col in columns:
        if col == "matching condition":
            if df[col][0] == "S2 nomatch,":
                values[col] = ["S2 nomatch"]
            else:
                values[col] = [df[col][0]]
        else: 
            values[col] = [df[col][0]]
    return pd.DataFrame(values)  
            

In [11]:
makeOneRow(all_frames[0])

,FP10,FP11,FP12,FP13,FP14,FP15,FP16,FP17,FP18,FP19,...,Y250,Y251,Y252,Y253,Y254,Y255,trial number,subject identifier,name,matching condition
0,-8.921,-8.433,-2.574,5.239,11.587,14.028,11.587,6.704,1.821,-1.109,...,3.153,6.571,12.431,15.849,16.337,14.872,0,a,co2a0000364,S1 obj


In [12]:
bigData = pd.concat([makeOneRow(df) for df in all_frames])
bigData.head()

AttributeError: 'list' object has no attribute 'head'

In [14]:
bigData = pd.concat(bigData)
bigData.head()

,FP10,FP11,FP12,FP13,FP14,FP15,FP16,FP17,FP18,FP19,...,Y250,Y251,Y252,Y253,Y254,Y255,trial number,subject identifier,name,matching condition
0,-8.921,-8.433,-2.574,5.239,11.587,14.028,11.587,6.704,1.821,-1.109,...,3.153,6.571,12.431,15.849,16.337,14.872,0,a,co2a0000364,S1 obj
0,11.434,9.481,6.551,3.133,1.180,1.180,1.668,2.157,2.645,4.110,...,0.020,-0.956,-2.421,-1.444,0.020,0.997,18,a,co2a0000364,S1 obj
0,3.947,4.924,4.924,4.435,3.947,3.459,2.970,1.994,1.017,-0.448,...,-7.406,-6.429,-4.476,-2.035,0.895,3.825,13,a,co2a0000369,S2 nomatch
0,-5.656,-6.632,-7.121,-9.074,-11.027,-12.492,-12.980,-13.468,-12.980,-12.492,...,-3.794,-1.353,0.600,1.577,2.065,2.553,14,a,co2a0000369,S1 obj
0,-0.773,-0.773,-1.750,-3.215,-3.703,-3.703,-3.703,-5.168,-7.609,-10.539,...,-21.088,-22.552,-22.064,-20.599,-19.135,-19.135,15,a,co2a0000369,S2 nomatch


In [16]:
s1Obj = bigData[(bigData["matching condition"] == "S1 obj")]
s1Obj.head()

,FP10,FP11,FP12,FP13,FP14,FP15,FP16,FP17,FP18,FP19,...,Y250,Y251,Y252,Y253,Y254,Y255,trial number,subject identifier,name,matching condition
0,-8.921,-8.433,-2.574,5.239,11.587,14.028,11.587,6.704,1.821,-1.109,...,3.153,6.571,12.431,15.849,16.337,14.872,0,a,co2a0000364,S1 obj
0,11.434,9.481,6.551,3.133,1.180,1.180,1.668,2.157,2.645,4.110,...,0.020,-0.956,-2.421,-1.444,0.020,0.997,18,a,co2a0000364,S1 obj
0,-5.656,-6.632,-7.121,-9.074,-11.027,-12.492,-12.980,-13.468,-12.980,-12.492,...,-3.794,-1.353,0.600,1.577,2.065,2.553,14,a,co2a0000369,S1 obj
0,3.082,2.106,0.641,-0.336,-0.824,-0.336,1.129,2.106,2.106,0.153,...,-0.448,1.017,1.506,2.482,3.459,3.947,16,a,co2a0000369,S1 obj
0,2.686,0.732,-0.732,-1.221,-1.709,-2.197,-4.639,-7.568,-10.498,-11.963,...,-0.600,-0.600,0.376,0.865,1.353,0.865,18,a,co2a0000369,S1 obj


In [25]:
dropColumns = ["trial number", "subject identifier", "name", "matching condition"]
s1X = s1Obj.drop(dropColumns, axis=1)
s1Y = s1Obj["subject identifier"]
s1LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr', max_iter=1000).fit(s1X, s1Y)

In [27]:
testDir = "data/SMNI_CMI_Test/Test/"
testFiles = os.listdir(testDir)
all_test = []
for name in testFiles:
    temp = pd.read_csv(testDir+name, header=0)
    all_test.append(temp)
bigTest = pd.concat([makeOneRow(df) for df in all_test])
bigTest.head()

,FP10,FP11,FP12,FP13,FP14,FP15,FP16,FP17,FP18,FP19,...,Y250,Y251,Y252,Y253,Y254,Y255,trial number,subject identifier,name,matching condition
0,-3.550,-5.015,-5.503,-3.550,-0.621,1.821,2.309,0.844,0.844,3.286,...,-2.411,1.007,3.937,3.937,2.472,1.495,30,a,co2a0000364,S1 obj
0,0.285,-0.203,0.285,2.726,4.679,5.656,5.656,5.656,4.191,1.750,...,4.252,6.205,8.647,9.623,8.158,4.252,50,a,co2a0000364,S1 obj
0,11.820,14.750,16.703,17.192,15.727,13.774,13.285,14.750,16.215,17.192,...,-2.991,-4.456,-5.432,-5.920,-6.409,-6.897,38,a,co2a0000369,S1 obj
0,-11.983,-15.401,-17.843,-18.819,-18.331,-16.866,-14.425,-12.960,-11.007,-8.565,...,-0.122,-1.587,-3.052,-5.005,-6.470,-7.935,43,a,co2a0000369,S2 nomatch
0,17.527,17.527,18.016,18.992,18.504,16.062,11.668,7.762,5.320,5.809,...,-5.971,-5.971,-6.460,-6.460,-6.948,-6.460,45,a,co2a0000369,S2 nomatch


In [28]:
s1Test = bigTest[(bigTest["matching condition"] == "S1 obj")]
s1X_test = s1Obj.drop(dropColumns, axis=1)
s1Y_test = s1Obj["subject identifier"]

In [33]:
s1Score = accuracy_score(s1LR.predict(s1X_test), s1Y_test)
print("Logistic Regression (S1 obj): {}".format(s1Score))

Logistic Regression (S1 obj): 1.0


In [39]:
def LRModel(df_train, matchCond, dropCols):
    temp = df_train[(df_train["matching condition"] == matchCond)]
    X = temp.drop(dropCols, axis=1)
    y = temp["subject identifier"]
    return LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr', max_iter=1000).fit(X, y)

In [41]:
def testModel(model, df_test, matchCond, dropCols):
    temp = df_test[(df_test["matching condition"] == matchCond)]
    X = temp.drop(dropCols, axis=1)
    y = temp["subject identifier"]
    return accuracy_score(model.predict(X), y)

In [40]:
LRmodels = {}
for cond in bigData["matching condition"].unique():
    LRmodels[cond] = LRModel(bigData, cond, dropColumns)

In [43]:
print("Logistic Regression Accuracy")
for cond, model in LRmodels.items():
    print("{}: {}".format(cond, testModel(model, bigTest, cond, dropColumns)))

Logistic Regression Accuracy
S1 obj: 0.7625
S2 nomatch: 0.83125
S2 match: 0.79375
